In [19]:
import cv2
import math
import serial
import time

# 영상 파일 또는 카메라 스트리밍 주소
url = 'http://192.168.0.114:81/stream'


# VideoCapture 객체 생성
cap = cv2.VideoCapture(url)

# 영상 출력 윈도우 생성
cv2.namedWindow('ESP32-CAM Stream')

# 영상의 가로, 세로 크기 확인
_, frame = cap.read()
height, width, _ = frame.shape

# 윈도우 크기 2배로 변경
window_width = width * 2
window_height = height * 2
cv2.resizeWindow('ESP32-CAM Stream', window_width, window_height)

# 각도 변수 초기화
phi, theta = 90, 0

# 시리얼 통신 객체 초기화
py_serial = serial.Serial(
    port='COM4',
    baudrate=9600,
)

# 중앙 십자 표시 함수
def draw_crosshair(frame, color):
    center_x = window_width // 2
    center_y = window_height // 2
    cv2.line(frame, (center_x - 10, center_y), (center_x + 10, center_y), color, 2)
    cv2.line(frame, (center_x, center_y - 10), (center_x, center_y + 10), color, 2)

# 클릭한 지점을 중앙에 오도록 이동하는 함수
def move_to_center(event, x, y, flags, param):
    global phi, theta
    
    if event == cv2.EVENT_LBUTTONDOWN:
        center_x = window_width // 2
        center_y = window_height // 2

        delta_x = center_x - x
        delta_y = center_y - y
        
        phi += math.degrees(math.asin(delta_x / center_x)) * (68/180) * (32/24)
        if phi<0:
            phi=0
        elif phi>180:
            phi=180
        theta += math.degrees(math.asin(delta_y / center_y))* (68/180)
        if theta<0:
            theta=0
        elif theta>180:
            theta=180

        command = f'{phi},{theta}\n'
        py_serial.write(command.encode())
        time.sleep(0.1)

        # phi와 theta 값을 출력
        print(f"ϕ: {phi}, θ: {theta}, x: {x}, y: {y}")

    # 윈도우에 중앙 십자 표시
    draw_crosshair(frame, (0, 255, 0))
    cv2.imshow('ESP32-CAM Stream', frame)

# 마우스 이벤트 콜백 함수 등록
cv2.setMouseCallback('ESP32-CAM Stream', move_to_center)

# 이미지 저장 관련 변수 초기화
save_interval = 2  # 저장 간격(초)
prev_save_time = time.time()

# 이미지 저장 경로
save_path = 'yolov3/img/'

# 영상 출력 반복
while True:
    # 프레임 수신
    ret, frame = cap.read()

    # 윈도우에 중앙 십자 표시
    draw_crosshair(frame, (0, 255, 0))
    
    # 프레임 크기 2배로 변경
    frame = cv2.resize(frame, (window_width, window_height))

    # 윈도우에 영상 표시
    cv2.imshow('ESP32-CAM Stream', frame)

    # 이미지 저장 ( esp cam으로 드론 저장하는 과정 )
    current_time = time.time()
    if current_time - prev_save_time >= save_interval:
    image_name = f'{save_path}drone_{int(current_time)}.jpg'  # 이미지 파일 경로를 지정
    cv2.imwrite(image_name, frame)
    print(f'Saved image: {image_name}')
    prev_save_time = current_time

    # 'q' 키를 누를 때까지 영상 표시 반복
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# VideoCapture 객체 해제
cap.release()

# 시리얼 통신 객체 해제
py_serial.close()

# 윈도우 창 닫기
cv2.destroyAllWindows()


ϕ: 122.9518548261245, θ: 0, x: 29, y: 304
ϕ: 148.40025646281794, θ: 7.932530761796806, x: 73, y: 154
ϕ: 154.211465457125, θ: 9.105546263485044, x: 256, y: 227
ϕ: 151.59242586709925, θ: 11.273677332674719, x: 349, y: 216
ϕ: 142.13470636173795, θ: 14.259333587546674, x: 423, y: 207
ϕ: 141.95432958522616, θ: 20.949035929223967, x: 322, y: 167
ϕ: 142.22489695197024, θ: 32.91259449167303, x: 317, y: 114
ϕ: 140.51032029783195, θ: 19.320225377071658, x: 339, y: 381
ϕ: 140.96127765403025, θ: 2.0255754941964987, x: 315, y: 412
ϕ: 141.77307491184757, θ: 0, x: 311, y: 393
ϕ: 142.31423343002598, θ: 18.631050919450157, x: 314, y: 58
ϕ: 153.31074007093443, θ: 11.370840759678478, x: 201, y: 319
ϕ: 142.60511766645695, θ: 14.72115121842111, x: 436, y: 203
ϕ: 156.07601032295625, θ: 9.808952801320286, x: 176, y: 294
ϕ: 142.30133356196538, θ: 15.46473606645614, x: 467, y: 178
ϕ: 143.29359485541207, θ: 28.945862513458028, x: 309, y: 100
ϕ: 153.99921725988955, θ: 29.126240203374667, x: 204, y: 238
ϕ: 144.92